# Filtering with PyClim



## 0. Installation of PyClim

 - svn checkout http://servforge.legi.grenoble-inp.fr/svn/soft-pyclim/trunk pyClim in /home/albert/lib/python

 - python setup.py install --home=~


## 1. Importing external modules

In [1]:
## magics

%load_ext version_information 
%version_information numpy,xarray,netCDF4


Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.5
numpy 1.11.1
xarray 0.9.1
netCDF4 1.2.4
Wed May 03 10:05:41 2017 CEST

In [1]:
## path for mdules

import sys
sys.path.insert(0,"/home/albert/lib/python")


In [2]:
## imports

import numpy as np
import dask
import xarray as xr
import GriddedData
import time

In [3]:
#- Other modules
import os
from glob import glob
import numpy.ma as ma
import scipy as sc
import WavenumberSpectrum as ws
import dask.array as da
from netCDF4 import Dataset

### palette
from matplotlib.colors import LogNorm
import matplotlib.cm as mplcm
import colormap as cmaps
import matplotlib.cm as cm

import seaborn as sns

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

## local/specific imports
import oocgcm
import oocgcm.filtering
import oocgcm.filtering.linearfilters as tf
%matplotlib inline

In [4]:
import pyClim as pc

## 2. Dataset

In [5]:
dir="/media/extra/DATA/NATL60/NATL60-CJM165-S/"
files03="NATL60-CJM165_y2013m03d*.1d_curloverf.nc"
files09="NATL60-CJM165_y2013m09d*.1d_curloverf.nc"

tfiles03=dir+files03
tfiles09=dir+files09

!ls $tfiles03
!ls $tfiles09

/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d01.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d02.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d03.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d04.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d05.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d06.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d07.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d08.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d09.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d10.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d11.1d_curloverf.nc
/media/extra/DATA/NATL60/NATL60-CJM165-S/NATL60-CJM165_y2013m03d12.1d_curloverf.nc
/med

## 4. 2D filtering

In [6]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf']
curl0 =curls[0]

In [7]:
curl0.shape

(3454, 5422)

In [8]:
asocurl0 = curl0[...,2000:,1500:].squeeze()
asocurl0.shape

(1454, 3922)

In [9]:
from pyClim.filTools import lanczos
n = 30
fc = 1./120 # cycle per observation
lowpass = lanczos('lp', [n,n],fc1=[fc, fc])


/home/albert/lib/python/pyClim/filTools.py:325: RuntimeWarning: divide by zero encountered in divide
  w_rect = 1 / z * fc_x * fc_y * spec.j1(2 * pi * z)
/home/albert/lib/python/pyClim/filTools.py:325: RuntimeWarning: invalid value encountered in multiply
  w_rect = 1 / z * fc_x * fc_y * spec.j1(2 * pi * z)
/home/albert/lib/python/pyClim/filTools.py:328: RuntimeWarning: invalid value encountered in divide
  np.sin(pi * k_y / n_y) / (pi * k_y /n_y ))


In [10]:
%time lpsocurl = lowpass.do_filtering2d(asocurl0)

CPU times: user 39.3 s, sys: 96 ms, total: 39.4 s
Wall time: 39.5 s


In [11]:
n = 120
fc = 1./120 # cycle per observation
lowpass = lanczos('lp', [n,n],fc1=[fc, fc])
%time lpsocurl = lowpass.do_filtering2d(asocurl0)

CPU times: user 9min 39s, sys: 120 ms, total: 9min 39s
Wall time: 9min 39s


In [12]:
%time lpsocurl = lowpass.do_filtering2d(curl0)

CPU times: user 32min 39s, sys: 188 ms, total: 32min 39s
Wall time: 32min 39s
